In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load your data
train_embeddings = torch.load('train_embeddings.pt').to(device)
test_embeddings = torch.load('test_embeddings.pt').to(device)
val_embeddings = torch.load('val_embeddings.pt').to(device)

train_labels = torch.load('train_labels_tensor.pt').to(device)
test_labels = torch.load('test_labels_tensor.pt').to(device)
val_labels = torch.load('val_labels_tensor.pt').to(device)


In [5]:
def normalize_embeddings(embeddings):
    norms = embeddings.norm(p=2, dim=1, keepdim=True)
    normalized_embeddings = embeddings / norms
    return normalized_embeddings

# Normalize the embeddings
train_embeddings= normalize_embeddings(train_embeddings).to(device)
val_embeddings = normalize_embeddings(val_embeddings).to(device)
test_embeddings = normalize_embeddings(test_embeddings).to(device)

# Now the embeddings are normalized and ready for use
print("Train embeddings shape:", train_embeddings.shape)
print("Validation embeddings shape:", val_embeddings.shape)
print("Test embeddings shape:", test_embeddings.shape)

Train embeddings shape: torch.Size([185542, 775])
Validation embeddings shape: torch.Size([23193, 775])
Test embeddings shape: torch.Size([23193, 775])


In [6]:
# Create DataLoader
train_dataset = TensorDataset(train_embeddings, train_labels)
val_dataset = TensorDataset(val_embeddings, val_labels)
test_dataset = TensorDataset(test_embeddings, test_labels)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class RNNWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, bidirectional=True):
        super(RNNWithAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.bidirectional_factor = 2 if bidirectional else 1

        # Input dimension should match combined embedding size (775)
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True)
        self.attention = nn.Linear(hidden_dim * self.bidirectional_factor, 1)
        self.fc = nn.Linear(hidden_dim * self.bidirectional_factor, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Debugging statement to ensure input shape is correct
        #print(f"Input shape: {x.shape}")

        # The input `x` must be of shape (batch_size, sequence_length, input_dim)
        rnn_out, _ = self.rnn(x)

        # Print shapes for debugging
        #print(f"rnn_out shape: {rnn_out.shape}")

        # Attention mechanism
        attn_weights = torch.tanh(self.attention(rnn_out))

        # Ensure attn_weights has the correct shape
        #print(f"attn_weights (before softmax) shape: {attn_weights.shape}")

        attn_weights = torch.softmax(attn_weights, dim=1)

        # Ensure attn_weights is 3D for batch matrix multiplication
        attn_weights = attn_weights.permute(0, 2, 1)
        #print(f"attn_weights (after softmax and unsqueeze) shape: {attn_weights.shape}")
        
        attn_output = torch.bmm(attn_weights, rnn_out).squeeze(1)

        # Final linear layer
        output = self.fc(attn_output)
        return self.sigmoid(output)


In [8]:
# Model parameters
input_dim = train_embeddings.shape[1]  # Assuming embeddings have shape (batch_size, sequence_length, embedding_dim)
hidden_dim = 128
output_dim = 1  # Binary classification

In [9]:

model = RNNWithAttention(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4 , weight_decay=1e-5)

In [13]:


# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    all_preds=[]
    all_labels=[]

    for inputs, labels in train_loader:
        # Reshape inputs to add the sequence dimension, if not already present
        inputs = inputs.unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print (f'Training precesion : {precision} , Training recall : {recall} , Training F1 : {f1}')

    # Validation step
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    val_preds=[]
    val_labels=[]


    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs=inputs.unsqueeze(1)
            outputs = model(inputs)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels.float())
            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = correct / total

    val_precision = precision_score(all_labels, all_preds)
    val_recall = recall_score(all_labels, all_preds)
    val_f1 = f1_score(all_labels, all_preds)

    #val_precision, val_recall, val_f1 = evaluate_model(model, val_loader, device)
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f"Validation Loss: {val_loss:.4f}")
    print(f'Accuracy: {val_accuracy:.4f} ,Validation Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}')





Epoch 1/10, Loss: 625.3525, Accuracy: 0.9152
Training precesion : 0.9225454824595072 , Training recall : 0.9065204038749582 , Training F1 : 0.9144627425403554
Epoch 1/10
Validation Loss: 77.9371
Accuracy: 0.9166 ,Validation Precision: 0.9225, Recall: 0.9065, F1 Score: 0.9145
Epoch 2/10, Loss: 607.7400, Accuracy: 0.9183
Training precesion : 0.9253675745557333 , Training recall : 0.9101410545144988 , Training F1 : 0.9176911584951786
Epoch 2/10
Validation Loss: 76.4868
Accuracy: 0.9174 ,Validation Precision: 0.9254, Recall: 0.9101, F1 Score: 0.9177
Epoch 3/10, Loss: 598.2844, Accuracy: 0.9196
Training precesion : 0.9257809510272362 , Training recall : 0.9123931854182605 , Training F1 : 0.9190383154238576
Epoch 3/10
Validation Loss: 75.3769
Accuracy: 0.9189 ,Validation Precision: 0.9258, Recall: 0.9124, F1 Score: 0.9190
Epoch 4/10, Loss: 591.1277, Accuracy: 0.9203
Training precesion : 0.925652302017744 , Training recall : 0.9140418745487656 , Training F1 : 0.9198104512085362
Epoch 4/10
Val

In [15]:
# Testing step
model.eval()
test_loss = 0
correct = 0
total = 0
all_preds=[]
all_labels=[]

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs=inputs.unsqueeze(1)
        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels.float())
        test_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = correct / total
print(f"Test Loss: {test_loss:.4f}")
test_precision = precision_score(all_labels, all_preds)
test_recall = recall_score(all_labels, all_preds)
test_f1 = f1_score(all_labels, all_preds)

# Test evaluation
#test_precision, test_recall, test_f1 = evaluate_model(model, test_loader, device)
print(f'Test Accuracy: {test_accuracy:.4f},Test Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}')

Test Loss: 70.2676
Test Accuracy: 0.9244,Test Precision: 0.9245, Recall: 0.9244, F1 Score: 0.9244
